In [1]:
import math
import pandas as pd
from scipy.optimize import minimize_scalar
import numpy as np
#data
L= 0.3 # module length in meters
n_f = 6 #number of fibers
i_d = 1.27e-3#internal diameter m
delta_membrane = 0.325e-3 #m
p_s = 0.1e-6 #pore size in meters
T = 298 #K
R = 8.314 #J/mol/K
F = 96485 #C/mol
e = 1.60217662e-19 #C
porosity = 0.6
permitivity = 6.375e-10 #permitivity of the medium C^2(J.m)^-1
I = 7.5 #ionic strength mM
gamma_w = 32400#shear rate s-1
phi_m = 0.68
eta_0 = 0.000905#Pa.second viscosity of hte TGM
rho_p = 1300 #kg/m3 density of the particles
rho_f = 1000 #kg/m3 density of the fluid
k_B = 1.38064852e-23 #J/mol/K Boltzmann constant
phi_w_guess = [0.64,0.64,0.64] #guess for the volume fraction of solute at the membrane wall
# Particle sizes (in meters)
particles = [
    {'name': '10nm', 'radius': 10e-9, 'phi_b': 0.01},
    {'name': '180nm', 'radius': 180e-9, 'phi_b': 0.05},
    {'name': '300nm', 'radius': 300e-9, 'phi_b': 0.06}
]
phi_w_dict={key:values for key, values in zip([particle['radius'] for particle in particles], phi_w_guess)}
target_species = 10e-9 
#equations
#Step 2 - evaluate the viscosity
    # no particle-particle interactions - dillute suspensions
def viscosity_no_PP(eta_0,phi_b,k1):
    """eta_0 - viscosity of the dispersion medium
    phi - volume in parts occupied by the dispersed solid
    k1 = shape factor"""
    if phi_b<0.1:
        eta_phi = eta_0*(1+5/2*phi_b)
    elif phi_b>0.1:
        eta_phi = eta_0*(1+5/2*phi_b+k1*phi_b**2)
    return eta_phi
  

eta_f = viscosity_no_PP(eta_0,sum(particles['phi_b'] for particles in particles),10)
# print(eta_f)  
diffusion_coeff = (k_B * T) / (6 * math.pi * eta_f * target_species)

Re = 1075
V_ax = Re*eta_f/(rho_f*i_d)
gamma_w = 8*V_ax/i_d
print(gamma_w,V_ax)

#k1 around 10 for spheres
#not sure about the shape factor

def max_agg_packing (phi_m):
    phi_Max=phi_m+0.74*(1-phi_m)
    return phi_Max
phi_M = max_agg_packing(phi_m)
# phi_M = 0.92

#step 3 - maximum back-transport velocity, u (m/s)- Brownian, shear-infuced and intertial lift - Full retention of all solutes
 #particle volume franction at the membrane wall for each particle size for the first iteration
#J_flux = 30 #LMH


def J_brownian(a, phi_w, phi_b):
    numerator = gamma_w * (k_B ** 2) * (T ** 2)
    denominator = (eta_f ** 2) * (a ** 2) * L
    term = (numerator / denominator) ** (1/3)
    J = 0.114 * term * math.log(phi_w / phi_b)
    return J

# Function to calculate J for shear-induced diffusion
def J_shear(a, phi_w, phi_b):
    term = (a ** 4 / L) ** (1/3)
    log_term = math.log(phi_w / phi_b)
    J = 0.078 * term * gamma_w * abs(log_term)
    return J

# Function to calculate J for inertial lift
def J_inertial(a):
    J = (0.036 * rho_p * (a ** 3) * (gamma_w ** 2)) / eta_f
    return J
#J solvent permeation flux (m/s)
pH =6.8
net_charge = {'10nm': 11.43, '180nm': -10.63, '300nm': -6.28}
# L_p =(porosity*(p_s)**2)/(8*eta_f*delta_membrane)
L_p= 60/3600/1000/1000

   
s = (( 5*eta_f* delta_membrane * L_p) /porosity)**(1/2)
k = (((permitivity*R*T)/(((F**2)*(2*I))))**(1/2))

for particle,charge in zip(particles,net_charge.values()):
    z = charge
    phi_b = particle['phi_b']
    a = particle['radius']
    lambda_aph = 1 - math.exp(-a/(2*s))
    
   
    delta_s = ((z)*e)/(4*math.pi*(a**2))

 
    eff_a = a + (((4*(a**3)*(delta_s**2))/(permitivity*k_B*T))*0.2*k)
 
    particle["radius"]=eff_a
target_species = particles[0]['radius']
phi_w_dict={key:values for key, values in zip([particle['radius'] for particle in particles], phi_w_guess)}
set_data = pd.DataFrame(columns=['name', 'radius', 'max_velocity', 'source'])
diffusion_coeff = (k_B * T) / (6 * math.pi * eta_f * target_species)

def min_flux(Particles, phi_w_list):
    """ have to put particles list with radius and phi_b,
    phi_w_list with phi_w
    results ai_target, J_flux and dataframe"""
    
    min_velocity = float('inf')
    selected_particle_radius = None
    velocities_list=[]

    for particle, phi_w in zip(Particles, phi_w_list):
        a = particle['radius']
        phi_b = particle['phi_b']
        
        # Calculate velocities
        brownian_velocity = J_brownian(a, phi_w, phi_b)
        shear_velocity = J_shear(a, phi_w, phi_b)
        inertial_velocity = J_inertial(a)
        
        # Find the maximum velocity
        max_velocity = max(brownian_velocity, inertial_velocity, shear_velocity)
        
        # Determine the source of the maximum velocity
        source = 'brownian' if max_velocity == brownian_velocity else 'inertial' if max_velocity == inertial_velocity else 'shear'
        
        # Update set_data by replacing previous row or adding a new one
        if particle['name'] in set_data['name'].values:
            set_data.loc[set_data['name'] == particle['name'], ['radius', 'max_velocity', 'source']] = [a, max_velocity, source]
        else:
            set_data.loc[len(set_data)] = [particle['name'], a, max_velocity, source]
        # Store all velocities in the DataFrame
        velocities_list.append({
            'name': particle['name'],
            'radius': a,
            'brownian_velocity': brownian_velocity,
            'shear_velocity': shear_velocity,
            'inertial_velocity': inertial_velocity
        })
        velocities_df=pd.DataFrame(velocities_list)
        # Check for the minimum velocity
        if max_velocity < min_velocity:
            min_velocity = max_velocity
            selected_particle_radius = a

    return selected_particle_radius, min_velocity, set_data,velocities_df

# Initialize variables for particles and inertial-lift properties
# ai_target, min_vel, dataf= min_flux(particles, phi_w_guess)

# Function to calculate φ_w for each particle size
def calculate_phi_w(Particles, a_target, min_velocity, data):
    count = 0
    # Dictionaries to store results
    phi_w_J_dict = {}
    phi_wjI_dict = {}
    inertial_particles = []
    
    for particle in Particles:
        a_particle = particle['radius']
        phi_b_particle = particle['phi_b']
        
        if a_particle != a_target:
            # Retrieve the velocity source from the data DataFrame
            source = data.loc[data['radius'] == a_particle, 'source'].values[0]
            
            # Use the source to choose the max_velocity
            if source == 'inertial':
                max_velocity = J_inertial(a_particle)
            elif source == 'shear':
                max_velocity = J_shear(a_particle, phi_w_dict.get(a_particle, 0.64), phi_b_particle)
            elif source == 'brownian':
                max_velocity = J_brownian(a_particle, phi_w_dict.get(a_particle, 0.64), phi_b_particle)
            else:
                raise ValueError(f"Unknown source {source} for particle {a_particle}")

            if max_velocity >= 10 * min_velocity and source == 'inertial':
                # Handle inertial particles
                phi_wjI = 0
                phi_wjI_dict[a_particle] = phi_wjI
                phi_w_dict[a_particle] = phi_wjI
                phi_w_J_dict[a_particle] = phi_wjI
                inertial_particles.append((a_particle, phi_b_particle, max_velocity))
            elif source == "inertial":
                count += 1
            elif source =="brownian" or source =="shear":
                # Define the objective function based on the source
                def objective_function(phi_w_particle):
                    # Recalculate velocities based on the current wall concentration phi_w_particle
                    if source == 'brownian':
                        max_velocity = J_brownian(a_particle, phi_w_particle, phi_b_particle)
                    elif source == 'shear':
                        max_velocity = J_shear(a_particle, phi_w_particle, phi_b_particle)
                    # elif source == 'inertial':
                    #     max_velocity = J_inertial(a_particle)
                    return abs(max_velocity - min_velocity)

                # Minimize the objective function to find the optimal phi_w for the particle
                result = minimize_scalar(objective_function, bounds=(0, 0.64), method='bounded')

                if result.success:
                    phi_w_optimal = result.x
                    phi_w_dict[a_particle] = phi_w_optimal
                    phi_w_J_dict[a_particle] = phi_w_optimal
                else:
                    print(f"Optimization failed for particle radius {a_particle:.1e} m")
    
    # Handle inertial particles after processing all particles
    if count == 1:
        for a, phi_b, u_j in inertial_particles:
            if phi_w_J_dict[a]!=0:
                phi_wjI = phi_M - sum(phi_w_J_dict.values())
                phi_wjI_dict[a] = phi_wjI
                phi_w_dict[a] = phi_wjI
    elif count > 1:
        total_u_j_inv = sum((phi_b / u_j) for _, phi_b, u_j in inertial_particles)
        phi_w_remaining = phi_M - sum(phi_w_J_dict.values())
        phi_w_jI_sum = sum((phi_b / u_j) / total_u_j_inv * phi_w_remaining for _, phi_b, u_j in inertial_particles)
        if abs(phi_M - (sum(phi_w_J_dict.values()) + phi_w_jI_sum)) > 1e-6:
            print("Warning: φ_M and the sum of φ_w_j do not match closely enough.")
        
        for a, phi_b, u_j in inertial_particles:
            if phi_w_J_dict[a]!=0:
                phi_wjI = (phi_b / u_j) / total_u_j_inv * phi_w_remaining
                phi_wjI_dict[a] = phi_wjI
                phi_w_dict[a] = phi_wjI

    return phi_w_dict


# Output the updated dictionary
#print(f"Updated phi_w_dict: {phi_w_dict}")
def packing_constraints(Particles, phi_w_list):
    ai_target, min_vel, dataf,vel= min_flux(Particles, phi_w_list)
    phi_w_updated=calculate_phi_w(Particles, ai_target, min_vel, dataf)
    
    tolerance = 1e-6  # Set a tolerance for convergence
    max_iterations = 1000

    for i in range(max_iterations):
        remaining_phiw_j = {a: phi_w for a, phi_w in phi_w_updated.items() if a != min(phi_w_updated.keys())}
        phi_w_i = phi_w_updated[ai_target] # Wall concentration of the target particle
        # Check if packing constraints are satisfied
        if (phi_M >= sum(phi_w_updated.values())) and (sum(remaining_phiw_j.values()) <= 0.68) and phi_w_updated[min(phi_w_updated.keys())] <= 0.74 * (1 - sum(remaining_phiw_j.values())):
            print(f"Iteration {i}: Packing constraints are met.")
            return phi_w_updated,   min_vel, ai_target, dataf
        
        # Store the previous values of phi_w for comparison
        phi_w_previous = phi_w_updated.copy()
        # Update phi_w for the target particle
        phi_w_i_corrected = phi_M * (phi_w_i / sum(phi_w_updated.values()))
        
        phi_w_updated[ai_target] = phi_w_i_corrected
        print(phi_w_updated)
        #print(phi_w_updated, J_flux, a_target)
        # Recalculate the flux and phi_w_j values after updating
        ai_target, min_vel, dataf,vel= min_flux(particles, phi_w_updated.values())
        phi_w_updated=calculate_phi_w(particles, ai_target, min_vel, dataf)
        # phi_smallest_int= phi_w_updated[min(phi_w_updated.keys())] / (1 -sum(phi_w for phi_w in phi_w_updated.values() if phi_w != phi_w_updated[min(phi_w_updated.keys())]))
#        
#       # #step - 9 - calculate the minimum
        # r_minimum = (min(phi_w_updated.keys()) * ( (math.sqrt(2) * ((( 4 * (4 / 3) * math.pi ) / phi_smallest_int )) ** (1/3)) - 2 ))/2
        
        # Check for the change in phi_w values between iterations
        phi_w_diff = sum(abs(phi_w_updated[a] - phi_w_previous[a]) for a in phi_w_updated)

        # If the change in phi_w values is smaller than the tolerance, stop the iteration
        # if phi_w_diff < tolerance:
        #     print(f"Iteration {i}: Will not converger further.")
        #     return phi_w_updated,   min_vel, ai_target, dataf
    # If the loop completes without finding a solution
    # print("Warning: No solution found within the iteration limit.")

  
    return phi_w_updated, min_vel, ai_target, dataf

def J_flux_calculation(Particles, phi_w_list):
    # Unpack the results from packing_constraints
    phi_w_updated, J_flux, ai_target, dataf = packing_constraints(Particles, phi_w_list)
    print(phi_w_updated)
    
    # Initialize J_pi with J_flux values from packing constraints for all particles
    J_pi = {a: J_flux for a in phi_w_updated.keys()}
    
    phi_w_dict_T = {}
    phi_w_dict_R = {}
    
    # Separate phi_w values into two dictionaries based on particle size comparison with p_s/2
    for (a, phi_w) in phi_w_updated.items():
        
        if a <= p_s / 2:
            phi_w_dict_T[a] = phi_w
            print(phi_w_dict_T)
        else:
            phi_w_dict_R[a] = phi_w
            print(phi_w_dict_R)

    # Update phi_w for particles in phi_w_dict_T based on ratio of min(phi_w_dict_R.keys()) / a
    for (a, phi_w) in phi_w_dict_T.items():
        if min(phi_w_dict_R.keys()) / a < 10:
            phi_w_updated[a] = 0.68 * (1 - sum(phi_w_dict_R.values()))
        else:
            phi_w_updated[a] = 0.74 * (1 - sum(phi_w_dict_R.values()))
    
    # Calculate J_pi only for particles in phi_w_dict_T
    for a , particle in zip(phi_w_dict_T.keys(), Particles):

        phi_b = particle['phi_b']
        
        # Calculate velocities
        brownian_velocity = J_brownian(a, phi_w_updated[a], phi_b)
        shear_velocity = J_shear(a, phi_w_updated[a], phi_b)
        inertial_velocity = J_inertial(a)
        
        # Update J_pi with the maximum velocity for particles in phi_w_dict_T
        max_velocity = max(brownian_velocity, inertial_velocity, shear_velocity)
        J_pi[a] = max_velocity

    return J_pi, phi_w_updated, J_flux, ai_target, dataf

# print(J_flux_calculation(particles, phi_w_guess))
#mass-transfer coefficient calculated from equation  1#step 10


def sieving_parameters(Particles, phi_w_list):
    J_pi,phi_w_updated, J_flux, ai_target, dataf = J_flux_calculation(Particles, phi_w_list)
    J_actual = J_flux*0.8
    S_oi_dep = {a: (1-(J_actual / J_pi[a])) for a in phi_w_updated.keys()}
    S_oi_membrane = {a: 0 for a in phi_w_updated.keys()}
    S_oi = {a: 0 for a in phi_w_updated.keys()}
    for particle,phi_w in zip(Particles,phi_w_updated.values()):
        phi_b = particle['phi_b']
        a = particle['radius']
        lambda_aph = 1 - math.exp(-a/(2*s))
        
   
    # for i in range(50):
        # Q=J_flux *n_f*math.pi*i_d*(L+L/20)
        # mass_transfer_k = 1.62 * (diffusion_coeff/i_d)*(((4*rho_f*Q)/(math.pi*n_f*i_d*eta_f))**(1/3))*((eta_f/(rho_f*diffusion_coeff))**(1/3))
        mass_transfer_k = J_pi[a]/math.log(phi_w/phi_b)
        print(f"mass transfer coefficient: {mass_transfer_k}")
        Sieving_coeff_int = ((1 - lambda_aph) ** 2) * (2 - ((1 - lambda_aph) ** 2)) * math.exp(-0.7146 * (lambda_aph ** 2))
        print(f"Initial Sieving coefficient: {Sieving_coeff_int}")
        # print(f"delta_m: {delta_m}")
        phi_e_K_d = (1 - lambda_aph) ** (9 / 2)
        print(f"phi_e_K_d: {phi_e_K_d}")
        # print  (f"cake_membrane_porosity: {porosity}")
        Peclet_m = ((J_actual * (delta_membrane)) / diffusion_coeff) * (Sieving_coeff_int / (porosity * phi_e_K_d))
        print  (f"Peclet_m: {Peclet_m}")
        Sieving_coeff_act = (Sieving_coeff_int * math.exp(Peclet_m)) / ((Sieving_coeff_int + math.exp(Peclet_m)) - 1)
        
        print(f"Actual Sieving coefficient: {Sieving_coeff_act}")
        Sieving_coeff_obs = Sieving_coeff_act / ((1 - Sieving_coeff_act) * math.exp(-J_actual / mass_transfer_k) + Sieving_coeff_act)
        print(f"Sieving coefficient: {Sieving_coeff_obs}")
        S_oi_membrane[a] = Sieving_coeff_obs
        S_oi[a] = S_oi_dep[a] * S_oi_membrane[a]
    print(S_oi)
    # yields = 1 - math.exp(-4 * (Sieving_coeff_obs))
    # print(f"Yields: {yields}")
            
    return dataf, phi_w_updated,J_flux 
     
print(sieving_parameters(particles, phi_w_guess))


6967.978175956351 1.1061665354330708
{1.2178470687840119e-08: np.float64(0.0492811824100591), 1.8010467748601438e-07: np.float64(0.6465767773786717), 3.000219208450732e-07: np.float64(0.21819341384560736)}
{1.2178470687840119e-08: np.float64(0.049280604399971756), 1.8010467748601438e-07: np.float64(0.6399900123285345), 3.000219208450732e-07: np.float64(0.22043818275320104)}
{1.2178470687840119e-08: np.float64(0.049280649592260396), 1.8010467748601438e-07: np.float64(0.6465755814231963), 3.000219208450732e-07: np.float64(0.21819145103327572)}
{1.2178470687840119e-08: np.float64(0.04927999746334027), 1.8010467748601438e-07: np.float64(0.6399771947623142), 3.000219208450732e-07: np.float64(0.22043993761333847)}
{1.2178470687840119e-08: np.float64(0.04927996531248603), 1.8010467748601438e-07: np.float64(0.646574049095913), 3.000219208450732e-07: np.float64(0.2181889290135261)}
{1.2178470687840119e-08: np.float64(0.04927921697561166), 1.8010467748601438e-07: np.float64(0.6399643771956641), 